In [218]:
include("Acor.jl")
include("generic.jl")
include("deshMethod.jl")
import Pkg
using Pkg, IJulia, Dates, DataFrames, Plots
using Distributions, Statistics, GLM, LinearAlgebra

make: Nothing to be done for `all'.


In [219]:
# Generates the path with steps number of steps. 
# autoreg parameters
a = 0.9
b = 2
steps = 10000

function get_next_step(previous::Float64) 
    w = rand(Normal(0,1))
    next_val = (a * previous) + (b * w)
end

function get_init_state() 
    init_sd = b^2 / (1-a^2)
    init_state = rand(Normal(0,init_sd))
end

function asim(start_state, step_function, iters = 1000) 
    curr_state = start_state
    path = Float64[]
    for i in 1:iters 
        path = push!(path,curr_state)
        curr_state = step_function(curr_state)
    end
    return path
end

start_state = get_init_state()
visited = asim(start_state, get_next_step, steps)

10000-element Vector{Float64}:
  5.674980084428475
  3.392407127481663
  3.922364863869984
  6.91672519164786
  7.628279329977709
  5.593198329584665
  4.276309086134731
  2.8631763342399426
 -0.2929336715759381
  1.1418534601205925
  1.0776647376530457
 -5.901807126566476
 -4.05991255768196
  ⋮
 -3.785498542541344
 -2.8620723815475992
 -4.153815486837695
 -5.160602118643634
 -2.5365157269250003
 -5.77047721790801
 -4.4891129414915705
 -1.9583741045486192
  1.315778136362684
  2.7224899737612107
 -0.3977550230016491
 -1.6629176852745642

In [186]:
M = sokal(calc_acfs, 5)
println(calc_t_int(M, calc_acfs))
exp_m = 1 + 2*(0.5/(1-0.5))
exp_m

2.3851810946605716


3.0

In [220]:
# for comparison between acor values and my values
my_time_sokal = Float64[]
acor_time = Float64[]

a= -0.8
for i in 1:10
    start_state = get_init_state()
    visited = asim(start_state, get_next_step, steps)
    calc_acfs = c_est_series(visited)
    println(mean(visited))
    maxLag = 100
    acor_sugg = Acor.acor(visited, maxLag)
    println(acor_sugg)
    acor_time = push!(acor_time, acor_sugg.tau)
    M = sokal(calc_acfs, 10)
    my_est = calc_t_int(M, calc_acfs)
    my_time_sokal = push!(my_time_sokal, my_est)
end

true_time = 1 + 2*(a/(1-a))
println(acor_time)
println(my_time_sokal)
1 + 2*((-0.8)/(1+0.8))

0.0025532063101446887
(mean = 0.002553206310144656, sigma = 0.01150876884900815, tau = 0.1207658678713186, status = 0)
-0.0029318014736090443
(mean = -0.0029318014736090378, sigma = 0.013161910367388433, tau = 0.16160955104560445, status = 0)
-0.017501099891248362
(mean = -0.01750109989124834, sigma = 0.010050156219114143, tau = 0.08970869535657704, status = 0)
0.0030436658176685684
(mean = 0.0030436658176685666, sigma = 0.012045305094996748, tau = 0.1377468479334454, status = 0)
0.007943280277258508
(mean = 0.007943280277258451, sigma = 0.012102592963358957, tau = 0.13739261998844654, status = 0)
0.001266984941315424
(mean = 0.0012669849413154213, sigma = 0.011775354430826274, tau = 0.12415397990040396, status = 0)
0.008656868141297661
(mean = 0.008656868141297673, sigma = 0.010450513101123902, tau = 0.10268671453653175, status = 0)
-0.009422916309000747
(mean = -0.009422916309000691, sigma = 0.013235982329397681, tau = 0.15846543819896794, status = 0)
-0.013261332097258008
(mean = -0

0.11111111111111105

In [221]:
# First, we get the simulations acfs
poss_as = [0.5, 0.7, 0.9, 0.99]
#poss_as = [-0.8]
steps = 10000
dict = Dict{String, Vector{Float64}}()

for a in poss_as
    states_visited = asim(get_init_state(), get_next_step, steps)
    acf_df = c_est_series(states_visited)
    dict[string(a)] = acf_df
end

old_ms = Float64[]
new_ts = Float64[]
ps = Int[]
qs = Int[]
as = Float64[]
real_t = Float64[]
rs = Float64[]
results_ar1s = Vector{DataFrame}()
poss_ps = [1, 2, 3, 4, 5]
poss_qs = [1, 2, 3, 4, 5]
poss_rs = [0]

# Now, we do the method. 
for a in poss_as
    acf = dict[string(a)]
    
    stop_search_index = findfirst(x -> x <= 0, acf)
    
    
    if stop_search_index !== nothing
        stop_search = 4 * stop_search_index
    else
        stop_search = length(acf)
    end

    n = 3
    #stop_search = min(max(stop_search, 10), 500)
    
    y = get_log(acf)[1:stop_search]
    x = collect(0:(stop_search-1))
    lim = length(y)
    
    results_ar1 = desh_method(x, y, lim, n)
    push!(results_ar1s, results_ar1)
end

19 milliseconds
0 milliseconds
0 milliseconds
0 milliseconds


In [222]:
for a in poss_as
    println(a)
    start_time = time()
    c = 4
    M_upper_bound = steps
    
    acf = dict[string(a)]
    
    m = sokal(acf, c)
    m_suggest = calc_t_int(m, acf)
    println(m)
    
    #stop_search = findfirst(x -> x <= 0, acf)
    #stop_search = stop_search_index !== nothing ? 1.5 * stop_search_index : length(acf)
    stop_search = m
    
    n = 3
    #stop_search = 100
    stop_search = stop_search < 10 ? 10 : stop_search
    
    y = get_log(acf)[1:stop_search]
    x = collect(0:(stop_search-1))
    lim = length(y)
    results_ar1 = desh_method(x, y, lim, n)
    
    for p in poss_ps
        for q in poss_qs
            if p <= q
                exp_m = 1 + 2 * (a / (1 - a))
                
                push!(real_m, exp_m)
                push!(old_ms, m_suggest)
                
                new_weights = get_desh_weights(results_ar1, p, q)
                vars_ar1 = get_slopes_sides(results_ar1, new_weights)
                
                idx = vars_ar1.x[findmax(vars_ar1.density)[2]]
                #display(plot(vars_ar1.x, vars_ar1.density))
                
                push!(ps, p)
                push!(qs, q)
                push!(as, a)
                push!(new_ts, idx)
                push!(real_t, exp_m)
            end
        end
    end

    # let p = 1 and q = 3
    end_time = time()
    println(end_time - start_time)
end

results = DataFrame("new_ts" => new_ts, "real_t" => real_t, "a" => as, "p" => ps, "qs"=>qs)
results

0.5
2
1 millisecond
0.29604411125183105
0.7
2
1 millisecond
0.2778041362762451
0.9
2
1 millisecond
0.27889299392700195
0.99
2
1 millisecond
0.2783341407775879


Row,new_ts,real_t
,Float64,Float64
1,8.27318,3.0
2,7.88152,3.0
3,5.66036,3.0
4,5.13457,3.0
5,4.86095,3.0
6,8.26781,3.0
7,7.91908,3.0
8,5.68718,3.0
9,5.17213,3.0
